In [41]:
%pip install asyncpg python-dotenv langchain-openai langchain langchain_community


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
import asyncpg
from dotenv import load_dotenv
import os
import sys
from langchain_openai import ChatOpenAI
from typing import List
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
import uuid

from models.models import ChatMessage
from prompt_template.prompt_templates import (
    DERIVE_GENDER_PREFERENCES_FROM_CONVERSATION
)

load_dotenv()


True

In [43]:
db_connection=await asyncpg.connect(os.getenv("DATABASE_URL"))

In [ ]:
# init LLM, I am using gpt-4o for this project
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [45]:
# GET USER NAME
try:
    query = '''
    select 
        distinct s.username
    from saachats as s
    order by s.username asc;
    '''

    result=await db_connection.fetch(
        query=query)
    print("Retrieve all username successfully")
except Exception as e:
    print("Failed when retrieve usernames")
    sys.exit()


username_arr=[] if not result else [user['username'] for user in result]

for i,username in enumerate(username_arr,1):
    print(f"User {i}: ",username)

Retrieve all username successfully
User 1:  [___]
User 2:  12345
User 3:  ADYUU
User 4:  aidentan
User 5:  AimingForTheSun
User 6:  akmoe
User 7:  aksharaa
User 8:  Alexhai
User 9:  alicelin
User 10:  ambanisupreme
User 11:  anahom
User 12:  Andrew222
User 13:  AneekC
User 14:  anonymous
User 15:  Austin
User 16:  Beellyyyy
User 17:  Breadcrumber101🍞
User 18:  Bryan
User 19:  bxla22
User 20:  Cap2014
User 21:  careylzh
User 22:  Chan mali chan
User 23:  chester
User 24:  ciw_gblk
User 25:  cloudrumbles
User 26:  cool03
User 27:  Dangerously adorable 
User 28:  dariell
User 29:  demonblaster208
User 30:  dexter
User 31:  Dhruv
User 32:  DHYEYA
User 33:  DicksonChew
User 34:  dnoma
User 35:  drlky
User 36:  dsadaf
User 37:  eajlucard
User 38:  eggcellent
User 39:  esannatraj
User 40:  etherealseven
User 41:  excel
User 42:  ffrixslee
User 43:  flappybird
User 44:  Funghai
User 45:  ganesh
User 46:  Gauahauah
User 47:  gaylelyn
User 48:  geraldek
User 49:  gizelle
User 50:  GranLim
User 5

In [46]:
# GET CONVERSATION
async def retrieve_conversation_of_specific_user_by_username(
    username:str):
    try:
        query = '''
        select *
            from saachats as s
        where s.username=$1;
        '''

        result=await db_connection.fetch(query,username)
        print("Retrieve user conversation successfully")
        return result
    except Exception as e:
        print("Failed when retrieve user conversation")
        sys.exit()

history_conversation=await retrieve_conversation_of_specific_user_by_username(
    username='dexter'
)

print("history_conversation: ",history_conversation)

def convert_to_list_of_chatmessages_object(chat_messages):
    user_chat_message=[ChatMessage(
                        username=message['username'],
                        timestamp=message['timestamp'],
                        content=message['content'],
                        role=message['role'],
                        category_id=message['category_id']
                        )
                        for message in chat_messages]
    return user_chat_message

user_chat_message=convert_to_list_of_chatmessages_object(chat_messages=history_conversation)
print("user chat messages: ",user_chat_message)


Retrieve user conversation successfully
history_conversation:  [<Record username='dexter' timestamp=datetime.datetime(2025, 2, 13, 8, 56, 55, 974118) content="Hi there! I'm Saa, your personal matchmaking assistant. I'm here to ask you some questions to get to know you better, and it should take around 15 minutes. Are you ready to start?" role='assistant' category_id=-1>, <Record username='dexter' timestamp=datetime.datetime(2025, 2, 13, 8, 57, 9, 489721) content='Yes' role='user' category_id=-1>, <Record username='dexter' timestamp=datetime.datetime(2025, 2, 13, 8, 57, 12, 761961) content='Great! To kick things off, could you share your gender and what gender preferences you have in a partner?' role='assistant' category_id=-1>, <Record username='dexter' timestamp=datetime.datetime(2025, 2, 13, 8, 57, 27, 771155) content='Make, preference females' role='user' category_id=-1>, <Record username='dexter' timestamp=datetime.datetime(2025, 2, 13, 8, 57, 30, 965090) content='Awesome, thanks f

In [47]:
def format_chat_history(
    chat_history: List[ChatMessage]):

    chat_history_formatted = ChatMessageHistory()
    
    for message in chat_history:
        if message.role == "user":
            chat_history_formatted.add_user_message(message.content)
        elif message.role == "assistant":
            # revert output format
            chat_history_formatted.add_ai_message(
                message=message.content
            )
        else:
            print("sender of this message is not valid")
            
    print("chat_history_formatted: ",chat_history_formatted)

    return chat_history_formatted

formatted_chat_history_test=format_chat_history(chat_history=user_chat_message)

chat_history_formatted:  AI: Hi there! I'm Saa, your personal matchmaking assistant. I'm here to ask you some questions to get to know you better, and it should take around 15 minutes. Are you ready to start?
Human: Yes
AI: Great! To kick things off, could you share your gender and what gender preferences you have in a partner?
Human: Make, preference females
AI: Awesome, thanks for sharing! Now, how old are you and what age range do you prefer in a partner?
Human: Im 21. I prefer someone about my age or younger
AI: Got it! Being 21 is such an exciting time. What do you envision for your future relationships and life goals?
Human: Elaborate please
AI: Sure! When I ask about your future relationships and life goals, I'm curious about what you hope to achieve in life, how you see a partner fitting into that vision, and what qualities or experiences you think are important for a fulfilling relationship. What are your thoughts on this?
Human: A family for sure, and a partner who’s willing 

In [48]:
def derive_basic_information_and_preferences(formatted_chat_history):
        question_generator_template = PromptTemplate.from_template(
                DERIVE_GENDER_PREFERENCES_FROM_CONVERSATION
        )

        chain = LLMChain(llm=llm, prompt=question_generator_template)
        
        desired_format="""
        {
                "gender": "Male",
                "age":21,
                "gender_preferences":"Female",
                "age_preferences":"18-25"
        }
        """

        response=chain(
                {"chat_history": formatted_chat_history,"desired_format":desired_format}, return_only_outputs=True
        )
        
       
        print("response: ",response['text'])

        return response['text']

user_information_and_preferences=derive_basic_information_and_preferences(
        formatted_chat_history=formatted_chat_history_test
)

# Remove triple backticks and possible "json" label
cleaned_json = user_information_and_preferences.strip("```").replace("json", "").strip()

# Load the cleaned JSON string
data = json.loads(cleaned_json)

print("data: ",data)

response:  ```json
{
    "gender": "Male",
    "age": 21,
    "gender_preferences": "Female",
    "age_preferences": "18-21"
}
```
data:  {'gender': 'Male', 'age': 21, 'gender_preferences': 'Female', 'age_preferences': '18-21'}


In [49]:
user_information_and_preferences_dict={}
mapping_username_and_random_ids={}

for username in username_arr:
    print("\n username: ",username,"\n")
    
    chat_messages_final=await retrieve_conversation_of_specific_user_by_username(
        username=username
    )
    
    formatted_chat_history_final=format_chat_history(
        chat_history=convert_to_list_of_chatmessages_object(
            chat_messages=chat_messages_final
        )
    )
    
    extracted_information=derive_basic_information_and_preferences(
        formatted_chat_history=formatted_chat_history_final
    )
    
    cleaned_json_string = extracted_information.strip("```").replace("json", "").strip()
    
    parsed_json_string=json.loads(cleaned_json_string)
    
    user_random_id = str(uuid.uuid4())  # Convert UUID to string
    
    mapping_username_and_random_ids[user_random_id]=username

    user_information_and_preferences_dict[user_random_id]=parsed_json_string
    
print("user : ",user_information_and_preferences)


 username:  [___] 

Retrieve user conversation successfully
chat_history_formatted:  Human: Haha
Human: Is this bot working?
Human: Hi
Human: Am I supposed to do a self introduction?
Human: How is this supposed to work?
response:  ```json
{
        "gender": "Null",
        "age": "Null",
        "gender_preferences": "Null",
        "age_preferences": "Null"
}
```

 username:  12345 

Retrieve user conversation successfully
chat_history_formatted:  AI: Hi there! I'm Saa, your personal matchmaking assistant. I'm here to ask you some questions to get to know you better and help find the perfect match for you. This conversation should take about 15 minutes. Are you ready to start?
Human: yes
AI: Awesome! To kick things off, can I ask what your gender is and what gender preferences you have in a partner?
Human: i am female, i prefer male
AI: Great! And how old are you, and what age range do you prefer in a partner?
Human: i am 19. i prefer the range 19-23
AI: Thanks for sharing that! Now

In [50]:
print("user_gender_preferences: ",user_information_and_preferences_dict)

user_gender_preferences:  {'f9bb40c0-39f3-48f5-b1ca-27c96baeb9c6': {'gender': 'Null', 'age': 'Null', 'gender_preferences': 'Null', 'age_preferences': 'Null'}, '371a71d5-9736-443d-b847-1c2d6aba6487': {'gender': 'Female', 'age': 19, 'gender_preferences': 'Male', 'age_preferences': '19-23'}, '2b3a8232-8fc4-417f-a348-159f4d9003b4': {'gender': 'Male', 'age': 23, 'gender_preferences': 'Female', 'age_preferences': '20-30'}, '886c9ceb-c702-4e34-a0ce-c506f6cf998f': {'gender': 'Female', 'age': 21, 'gender_preferences': 'Male', 'age_preferences': '21-26'}, '39bc7c6c-9fd2-46be-add1-be1a36bac01b': {'gender': 'Male', 'age': 23, 'gender_preferences': 'Female', 'age_preferences': '18-25'}, 'f0d7601b-ec63-4d40-b2d2-8c89fd14f231': {'gender': 'Male', 'age': 26, 'gender_preferences': 'Female', 'age_preferences': '20 and above'}, 'daa4250c-73f9-48e0-931d-58b0e0f3f272': {'gender': 'Female', 'age': 18, 'gender_preferences': 'Male', 'age_preferences': '22'}, '221eb437-981d-44bb-a577-00324b3dc7d6': {'gender': 

In [51]:
# Write dictionary to JSON file
import json 

json_filename_1 = "extracted_user_info.json"
with open(json_filename_1, "w") as json_file:
    json.dump(user_information_and_preferences_dict, json_file, indent=4)

print(f"JSON file '{json_filename_1}' created successfully!")

json_filename_2 = "username_mapping.json"
with open(json_filename_2, "w") as json_file:
    json.dump(mapping_username_and_random_ids, json_file, indent=4)
    
print(f"JSON file '{json_filename_2}' created successfully!")

JSON file 'extracted_user_info.json' created successfully!
JSON file 'username_mapping.json' created successfully!
